In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import copy
from hand_classifier.utils import *

In [ ]:
%matplotlib inline
from object_detection.darkflow.darkflow.net.build import TFNet

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Softmax
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = tf.keras.models.load_model('hand_classifier/Deep-Learning/Transfer Learning CNN/mobilenet_many_angles_new4.h5')
gt_path = 'hand_classifier/frames/curl_annotations'
frames_path = 'hand_classifier/frames/curl'
video_path = 'test_video.avi'

In [ ]:
options = {
    'gpu': 1.0,
    'model': 'object_detection/darkflow/cfg/yolov2-tiny-voc-hand.cfg',
    'load': 56440,
    'threshold': 0.1,
    'backup': 'E:/ar_fps/object_detection/ckpt',
}
detector = TFNet(options)
detector.load_from_ckpt()

In [ ]:
# img = cv2.imread('cropped/img98.jpg')
# img = cv2.resize(img, (75, 75))
# batch = np.expand_dims(img, axis=0).astype(np.float32)
# res = model.predict(batch)
# print(res)

In [ ]:
def read_xml(xml_file):
    boxes = []
    classes = []
    data = ET.parse(xml_file)
    root = data.getroot()
    objs = root.findall('object')
    for obj in objs:
        box = obj.find('bndbox')
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        box = [xmin, ymin, xmax, ymax]
        boxes.append(box)
        
#         name = root.find('class_name') # TEMPORARY; made a bug by mistake
        name = obj.find('class_name')
        if name is None:
            name = obj.find('name')
        classes.append(name.text)
    return boxes, classes

In [ ]:
# detect graph
# ckpt_path = "handtracking/handtracking/hand_inference_graph/frozen_inference_graph.pb "
# detection_graph = tf.Graph()
# with detection_graph.as_default():
#     od_graph_def = tf.compat.v1.GraphDef()
#     with tf.io.gfile.GFile(ckpt_path, 'rb') as fid:
#         serialized_graph = fid.read()
#         od_graph_def.ParseFromString(serialized_graph)
#         tf.import_graph_def(od_graph_def, name='')
#     sess = tf.compat.v1.Session(graph=detection_graph)


frames = []
# ground truth boxes 

# video
# cap = cv2.VideoCapture(video_path)  
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     frames.append(frame)
# cap.release()

# from directory
frame_files = os.listdir(frames_path)
for frame_file in frame_files:
    frame = cv2.imread(os.path.join(frames_path, frame_file))
    frames.append(frame)

In [ ]:
src_dir = 'E:/ar_fps/action_perception/frames'
# frames = os.listdir('test')
labels = get_class_labels()
gt_files = os.listdir(gt_path)
failed_dir = 'detect_classify_images/failed/'
train_dir = 'hand_classifier/images/train'
train_class_dir = os.path.join(train_dir, os.path.basename(frames_path))
# num_train = len(os.listdir(train_class_dir))
# offset = num_train
num_train = 0
offset = 0
assets_dir = 'C:/../Garland\'s Pixel/Internal shared storage/ar_fps'
asset_img_path = os.path.join(assets_dir, 'test.jpg')

In [ ]:
print(len(frames))
print(offset)
print(get_class_labels())

In [ ]:
import time
total_time = 0
write_img = 0

In [ ]:
for ind, image in enumerate(frames):
    
    # detect with ground truth
    gt_file = os.path.join(gt_path, gt_files[ind])
    boxes, classes = read_xml(gt_file)
    left, top, right, bottom = boxes[0]
    act_left, act_top, act_right, act_bottom = boxes[0]
    actual_label = classes[0]

    cropped = image[top:bottom, left:right]
    
    start_time = time.time()
    
    # detect with handtracking object detector
#     image = cv2.imread(os.path.join(src_dir, image_name))
#     image_inp = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     im_height, im_width, _ = image_inp.shape

    # plt.imshow(image)

#     image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
#     detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
#     detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
#     detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
#     num_detections = detection_graph.get_tensor_by_name('num_detections:0')

#     batch = np.expand_dims(image_inp, axis=0) 

#     boxes, scores, classes, num = sess.run(
#       [detection_boxes, detection_scores, detection_classes, num_detections],
#       feed_dict={image_tensor: batch}
#     )

#     boxes = np.squeeze(boxes)
#     scores = np.squeeze(scores)

#     i = np.argmax(scores)
#     left = int(boxes[i][1] * im_width)
#     right = int(boxes[i][3] * im_width)
#     top = int(boxes[i][0] * im_height)
#     bottom = int(boxes[i][2] * im_height)


    # detect with actual object detector
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = detector.return_predict(img)

    
    # send to android app
    start_write_img = time.time()
    cv2.imwrite(asset_img_path, cropped)
#     cv2.imwrite('cropped/img{}.jpg'.format(str(ind)), cropped)
    end_write_img = time.time()
    write_img += end_write_img - start_write_img
    
    # wait til get response
    print('wait til get response')
    
    # classifier
#     img = cv2.resize(cropped, (75, 75))
#     batch = np.expand_dims(img, axis=0).astype(np.float32)
#     res = model.predict(batch)
#     res_ind = np.argmax(res[0])
# #     print(res_ind)
#     name = labels[res_ind]
    
#     p1 = (int(left), int(top))
#     p2 = (int(right), int(bottom))
#     disp = np.copy(image)
#     cv2.rectangle(disp, p1, p2, (77, 255, 9), 3, 1)
# #     cv2.putText(disp, "score: {}".format(scores[i]), p1, cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 1)
#     cv2.putText(disp, name, (left, top + 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 1)
#     cv2.imwrite('detect_classify_images/output/img{}.jpg'.format(str(ind + offset)), disp)
    
#     # isolate failed images to train on again 
#     if name != actual_label:
#         print('{}: prediction is {} | actual is {}'.format(ind, name, actual_label))
#         failed_class_dir = os.path.join(failed_dir, actual_label)
#         actual_box = image[act_top:act_bottom, act_left:act_right]
#         if not os.path.exists(failed_class_dir):
#             os.makedirs(failed_class_dir)
            
#         cv2.imwrite(os.path.join(failed_class_dir, 'failed_img{}.jpg'.format(str(ind + offset))), actual_box)
    end_time = time.time()
    total_time += end_time - start_time

In [ ]:
print('Avg write time: {}'.format(write_img / len(frames)))

In [ ]:
# live demo
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    
    crop = frame[200:325, 200:325].copy()
    cv2.rectangle(frame, (200, 200), (275, 325), (0, 255, 0), 3, 1)
    img = cv2.resize(crop, (75, 75))
    batch = np.expand_dims(img, axis=0).astype(np.float32)
    res = model.predict(batch)
    name = labels[np.argmax(res[0])]
#     print(name)
    
    cv2.putText(frame, "{} | {}".format(name, res), (40, 40), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 0), 1)
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
import sys
import subprocess
import random
import time
import threading
from queue import *
class AsynchronousFileReader(threading.Thread):
    def __init__(self, fd, my_queue):
        assert isinstance(my_queue, Queue)
        assert callable(fd.readline)
        threading.Thread.__init__(self)
        self._fd = fd
        self._queue = my_queue
    
    def run(self):
        for line in iter(self._fd.readline, ''):
            self._queue.put(line)
    
    def eof(self):
        return not self.is_alive() and self._queue.empty()
